In [2]:
import os
from dotenv import load_dotenv
import numpy as np
import tensorflow as tf
import pandas as pd
import datetime


2023-06-10 19:25:59.055040: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-10 19:25:59.490417: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [8]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

dataset = tf.data.Dataset.from_tensor_slices((x_train,y_train))
datasetVal = tf.data.Dataset.from_tensor_slices((x_test,y_test))


In [17]:
from tensorflow.keras import layers

def createModel():
    inputs = tf.keras.Input(shape=(28,28))
    # x=layers.Embedding(VOCAB_SIZE, EMBED_DIM, name="embedding")(inputs)

    x=layers.Conv2D(8, 3, activation=tf.keras.activations.relu)(tf.expand_dims(inputs,3))
    x=layers.Conv2D(8, 3, activation=tf.keras.activations.relu)(x)

    x=layers.Flatten()(x)
    x=layers.Dense(32)(x)
    x=layers.LeakyReLU()(x)
    outputs=layers.Dense(1, activation=tf.keras.activations.softmax)(x)
    # outputs=layers.Softmax()(x)

    return tf.keras.Model(inputs=inputs, outputs=outputs, name="embedConvModel")

model = createModel()

model.summary()


Model: "embedConvModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 28, 28)]          0         
                                                                 
 tf.expand_dims_5 (TFOpLambd  (None, 28, 28, 1)        0         
 a)                                                              
                                                                 
 conv2d_14 (Conv2D)          (None, 26, 26, 8)         80        
                                                                 
 conv2d_15 (Conv2D)          (None, 24, 24, 8)         584       
                                                                 
 flatten_3 (Flatten)         (None, 4608)              0         
                                                                 
 dense_4 (Dense)             (None, 32)                147488    
                                                    

In [29]:


BATCH_SIZE=64
LOG_INTERVAL=20
epochs = 30

# Instantiate an optimizer 
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-4)

# Instantiate a loss function.
loss_fn = tf.keras.losses.BinaryCrossentropy()



batchedDataset = dataset.batch(BATCH_SIZE, drop_remainder=False)
batchedDatasetVal = datasetVal.batch(BATCH_SIZE, drop_remainder=False)

# batchedDataset = batchedDataset.cache(os.path.join(DATA_PATH, "datasetCache"+SO))
# batchedDatasetVal = batchedDatasetVal.cache(os.path.join(DATA_PATH, "datasetCacheVal"+SO))

@tf.function()
def trainStep(x_batch_train, y_batch_train):
    with tf.GradientTape() as tape:

        probs = model(x_batch_train, training=True) 
        loss_value = loss_fn(y_batch_train, probs)

    grads = tape.gradient(loss_value, model.trainable_weights)
    #Gradient clipping
    # grads = [tf.clip_by_norm(g, 2.0) for g in grads]

    optimizer.apply_gradients(zip(grads, model.trainable_weights)) 
    return loss_value, probs

@tf.function()
def valStep(x_batch_val, y_batch_val):
    valProbs = model(x_batch_val, training=False)


maxStep=0

for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch+1,))
    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(batchedDataset):
        loss_value, probs=trainStep(x_batch_train,y_batch_train)
        print("train step at step ",step)
        # Log 
        if step % LOG_INTERVAL == 0:
            print("Log at step", step)

    print("Epoch finished. Start validation")
    for x_batch_val, y_batch_val in batchedDatasetVal:
        valStep(x_batch_val, y_batch_val)



Start of epoch 1


2023-06-10 19:20:41.695820: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-10 19:20:42.127878: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-06-10 19:20:43.188458: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x53f74de0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-06-10 19:20:43.188497: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce GTX 1070, Compute Capability 6.1
2023-06-10 19:20:43.191326: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-06-10 19:20:43.284864:

train step at step  0
Log at step 0
train step at step  1
train step at step  2
train step at step  3
train step at step  4
train step at step  5
train step at step  6
train step at step  7
train step at step  8
train step at step  9
train step at step  10
train step at step  11
train step at step  12
train step at step  13
train step at step  14
train step at step  15


KeyboardInterrupt: 